# Tracing for Different Types of Runs

Types of runs:
- LLM: Invokes an LLM
- Retriever: Retrieves documents from databases or other sources
- Tool: Executes actions with function calls
- Chain: Default type; combines multiple Runs into a larger process
- Prompt: Hydrates a prompt to be used with an LLM
- Parser: Extracts structured data

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

LLM run:

In [5]:
from langsmith import traceable

inputs = [
  {"role": "system", "content": "You are a librarian who helps people locate their favourite books in the library based on the genre or theme or writer of said book."},
  {"role": "user", "content": "I'm looking for a book written in the dystopian genre."},
]

output = {
  "choices": [
      {
          "message": {
              "role": "assistant",
              "content": "What author are you looking for? When was the book written?"
          }
      }
  ]
}

@traceable(
  run_type="llm",
  metadata={
    "ls_provider": "openai",
    "ls_model_name": "gpt-4"
  }
)
def chat_model(messages: list):
  return output

chat_model(inputs)

{'choices': [{'message': {'role': 'assistant',
    'content': 'What author are you looking for? When was the book written?'}}]}

### Handling Streaming LLM Runs

In [7]:
def _reduce_chunks(chunks: list):
    all_text = "".join([chunk["choices"][0]["message"]["content"] for chunk in chunks])
    return {"choices": [{"message": {"content": all_text, "role": "assistant"}}]}

@traceable(
    run_type="llm",
    metadata={"ls_provider": "my_provider", "ls_model_name": "my_model"},
    reduce_fn=_reduce_chunks,
)
def my_streaming_chat_model(messages: list):
    for chunk in ["Hello! Please explain this gap in your resume, " + messages[1]["content"]]:
        yield {
            "choices": [
                {
                    "message": {
                        "content": chunk,
                        "role": "assistant",
                    }
                }
            ]
        }

list(
    my_streaming_chat_model(
        [
            {"role": "system", "content": "You are an annoying interviewer. Please question the user."},
            {"role": "user", "content": "job applicant"},
        ],
    )
)

[{'choices': [{'message': {'content': 'Hello! Please explain this gap in your resume, job applicant',
     'role': 'assistant'}}]}]

### Retriever Runs + Documents

In [8]:
from langsmith import traceable

def _convert_docs(results):
  return [
      {
          "page_content": r,
          "type": "Document",
          "metadata": {"library_section": "Dystopian Novels"}
      }
      for r in results
  ]

@traceable(
    run_type="retriever"
)
def retrieve_docs(query):
  contents = ["1984 by George Orwell", "Fahrenheit 451 by Ray Bradbury", "Brave New World by Aldous Huxley"]
  return _convert_docs(contents)

retrieve_docs("Searching for dystopian novels:")

[{'page_content': '1984 by George Orwell',
  'type': 'Document',
  'metadata': {'library_section': 'Dystopian Novels'}},
 {'page_content': 'Fahrenheit 451 by Ray Bradbury',
  'type': 'Document',
  'metadata': {'library_section': 'Dystopian Novels'}},
 {'page_content': 'Brave New World by Aldous Huxley',
  'type': 'Document',
  'metadata': {'library_section': 'Dystopian Novels'}}]

### Tool Calling

In [11]:
from langsmith import traceable
from openai import OpenAI
from typing import List, Optional
import json

openai_client = OpenAI()

@traceable(
  run_type="tool"
)
def get_current_temperature(location: str, unit: str):
    return 65 if unit == "Fahrenheit" else 17

@traceable(run_type="llm")
def call_openai(
    messages: List[dict], tools: Optional[List[dict]]
) -> str:
  return openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
    tools=tools
  )

@traceable(run_type="chain")
def ask_about_the_weather(inputs, tools):
  response = call_openai(inputs, tools)
  tool_call_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
  location = tool_call_args["location"]
  unit = tool_call_args["unit"]
  tool_response_message = {
    "role": "tool",
    "content": json.dumps({
        "location": location,
        "unit": unit,
        "temperature": get_current_temperature(location, unit),
    }),
    "tool_call_id": response.choices[0].message.tool_calls[0].id
  }
  inputs.append(response.choices[0].message)
  inputs.append(tool_response_message)
  output = call_openai(inputs, None)
  return output

tools = [
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., Chennai, TN"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          "required": ["location", "unit"]
        }
      }
    }
]
inputs = [
  {"role": "system", "content": "You are an assistant for question-answering tasks."},
  {"role": "user", "content": "What is the weather today in Chennai, India?"},
]

ask_about_the_weather(inputs, tools)

ChatCompletion(id='chatcmpl-CMzSUHZE3Y2gljVRzlpTWtIw32saj', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The current temperature in Chennai, India, is 17°C. Please note that weather conditions can change rapidly, so it's a good idea to check a reliable weather service for the latest updates.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1759595498, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_51db84afab', usage=CompletionUsage(completion_tokens=38, prompt_tokens=85, total_tokens=123, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))